In [139]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from time import sleep
from urllib.error import HTTPError
import re
import json

team_names = {
    'ATL': 'ATL',
    'BOS': 'BOS',
    'BKN': 'BRK',
    'CHA': 'CHO',
    'CHI': 'CHI',
    'CLE': 'CLE',
    'DAL': 'DAL',
    'DEN': 'DEN',
    'DET': 'DET',
    'HOU': 'HOU',
    'IND': 'IND',
    'GS': 'GSW',
    'LAC': 'LAC',
    'LAL': 'LAL',
    'MEM': 'MEM',
    'MIA': 'MIA',
    'MIL': 'MIL',
    'MIN': 'MIN',
    'NO': 'NOP',
    'NY': 'NYK',
    'OKC': 'OKC',
    'ORL': 'ORL',
    'PHI': 'PHI',
    'PHX': 'PHO',
    'POR': 'POR',
    'SAC': 'SAC',
    'SA': 'SAS',
    'TOR': 'TOR',
    'UTAH': 'UTA',
    'WSH': 'WAS',
}


def process_game(soup, team):

    return


def scrape(home, away, d, game_number):
    day = d.strftime('%d')
    month = d.strftime('%m')
    year = d.strftime('%y')
    date = d.strftime('%Y-%m-%d')
    print(home, away, date)
    gameid = 21600000 + game_number
    gameid_str = '00%d' % gameid
    try:
        url = 'http://stats.inpredictable.com/nba/wpBox.php?season='+ year + '&month=' + month + '&date=' + date + '&gid=' + gameid_str
        url_page = urlopen(url)
    except HTTPError:
        sleep(10)
        url_page = urlopen(url)
    soup = BeautifulSoup(url_page, "html.parser")
    
    ### GET GAME excitement, comeback, mvp, lvp
    score_data = str(soup.find("div", {"id": "score"}))
    
    exc_text = 'Excitement</b></td><td>'
    exc_idx = score_data.index(exc_text)
    excitement_score = score_data[exc_idx + len(exc_text): exc_idx + len(exc_text) + 3]
    
    cb_text = 'Comeback</b></td><td>'
    cb_idx = score_data.index(cb_text)
    comeback_score = score_data[cb_idx + len(cb_text): cb_idx + len(cb_text) + 3]
    
    split = score_data.split('MVP</b></td><td>')[1]
    mvp = split.split('</td></tr>')[0]
    
    split = score_data.split('LVP</b></td><td>')[1]
    lvp = split.split('</td></tbody></table></div>')[0]
        
    ### GET EVENTS ###
    script = str(soup.findAll('script')[8])    
    split = script.split('"rows": ')[1]
    split = '{"rows": ' + split
    
    event_data = split.split('data_wprb.setColumnProperty')[0]
    event_data = event_data[:-23]
    events = []
    event_data = event_data.split('Q')
    for event in event_data:
        event = 'Q' + event
        event = event.replace('<u>', '').replace('</u>', '')
        event = event.replace('<b>', '').replace('</b>', '')
        event_split = event.split('<br />')
        if len(event_split) > 5:
            winprob_swing = event_split[2][-5:].lstrip()
            event = (event_split[1], winprob_swing)
            events.append(event)

    processed_game = "{},{},{},{},{},{},{},{}".format(home, away, d.strftime('%Y%m%d'), excitement_score, comeback_score, mvp, lvp, events)
    return processed_game
    
def process_year(year):

    game_list = open('data/'+year+'/'+year+'_articles.csv', 'r')
    file = open('data/'+year+'/'+year+'_play_by_play.csv', 'a')
    
    game_number = 0
    i = 0
    for game in game_list:
        i += 1
        tokens = game.split(',')
        try:
            game_number += 1
            away = team_names[tokens[1]]
            home = team_names[tokens[2]]
            d = datetime.strptime(tokens[3], "%Y%m%d")
            game_data = scrape(home, away, d, game_number)
            file.write(game_data)
            sleep(3)
            file.write('\n')
        except KeyError:
            print("error")
            game_number -= 1
            continue
    game_list.close()
    file.close()

process_year('2016')

error
CLE BOS 2017-10-17
GSW HOU 2017-10-17
WAS PHI 2017-10-18
DET CHO 2017-10-18
IND BRK 2017-10-18
ORL MIA 2017-10-18
BOS MIL 2017-10-18
MEM NOP 2017-10-18
DAL ATL 2017-10-18
UTA DEN 2017-10-18
SAS MIN 2017-10-18
PHO POR 2017-10-18
SAC HOU 2017-10-18
TOR CHI 2017-10-19
OKC NYK 2017-10-19
LAL LAC 2017-10-19
MIL CLE 2017-10-20
CHO ATL 2017-10-20
IND POR 2017-10-20
PHI BOS 2017-10-20
WAS DET 2017-10-20
BRK ORL 2017-10-20
MIN UTA 2017-10-20
DAL SAC 2017-10-20
NOP GSW 2017-10-20
PHO LAL 2017-10-20
TOR PHI 2017-10-21
CLE ORL 2017-10-21
MIA IND 2017-10-21
NYK DET 2017-10-21
CHI SAS 2017-10-21
HOU DAL 2017-10-21
MEM GSW 2017-10-21
MIL POR 2017-10-21
DEN SAC 2017-10-21
UTA OKC 2017-10-21
LAC PHO 2017-10-21
BRK ATL 2017-10-22
OKC MIN 2017-10-22
LAL NOP 2017-10-22
DET PHI 2017-10-23
MIA ATL 2017-10-23
HOU MEM 2017-10-23
MIL CHO 2017-10-23
DAL GSW 2017-10-23
SAS TOR 2017-10-23
DEN WAS 2017-10-23
PHO SAC 2017-10-23
CLE CHI 2017-10-24
ORL BRK 2017-10-24
BOS NYK 2017-10-24
MIN IND 2017-10-24
POR NO

MIA LAC 2017-12-16
HOU MIL 2017-12-16
MIN PHO 2017-12-16
SAS DAL 2017-12-16
MEM BOS 2017-12-16
TOR SAC 2017-12-17
DET ORL 2017-12-17
BRK IND 2017-12-17
WAS CLE 2017-12-17
CHO NYK 2017-12-18
IND BOS 2017-12-18
ATL MIA 2017-12-18
CHI PHI 2017-12-18
HOU UTA 2017-12-18
MIN POR 2017-12-18
OKC DEN 2017-12-18
DAL PHO 2017-12-18
SAS LAC 2017-12-18
LAL GSW 2017-12-18
PHI SAC 2017-12-19
WAS NOP 2017-12-19
MIL CLE 2017-12-19
CHO TOR 2017-12-20
ATL IND 2017-12-20
BOS MIA 2017-12-20
BRK SAC 2017-12-20
CHI ORL 2017-12-20
HOU LAL 2017-12-20
OKC UTA 2017-12-20
DAL DET 2017-12-20
DEN MIN 2017-12-20
POR SAS 2017-12-20
GSW MEM 2017-12-20
LAC PHO 2017-12-20
CLE CHI 2017-12-21
PHI TOR 2017-12-21
NYK BOS 2017-12-21
PHO MEM 2017-12-21
UTA SAS 2017-12-21
DET NYK 2017-12-22
ORL NOP 2017-12-22
BRK WAS 2017-12-22
MIA DAL 2017-12-22
HOU LAC 2017-12-22
MIL CHO 2017-12-22
OKC ATL 2017-12-22
POR DEN 2017-12-22
GSW LAL 2017-12-22
TOR PHI 2017-12-23
CHO MIL 2017-12-23
IND BRK 2017-12-23
WAS ORL 2017-12-23
ATL DAL 2017

MIL DEN 2018-02-15
MIN LAL 2018-02-15
CHO BRK 2018-02-22
ORL NYK 2018-02-22
CLE WAS 2018-02-22
CHI PHI 2018-02-22
SAC OKC 2018-02-22
GSW LAC 2018-02-22
DET BOS 2018-02-23
IND ATL 2018-02-23
WAS CHO 2018-02-23
TOR MIL 2018-02-23
HOU MIN 2018-02-23
MEM CLE 2018-02-23
NOP MIA 2018-02-23
DEN SAS 2018-02-23
PHO LAC 2018-02-23
UTA POR 2018-02-23
LAL DAL 2018-02-23
PHI ORL 2018-02-24
MIA MEM 2018-02-24
NYK BOS 2018-02-24
GSW OKC 2018-02-24
MIN CHI 2018-02-24
PHO POR 2018-02-24
UTA DAL 2018-02-24
SAC LAL 2018-02-24
CHO DET 2018-02-25
MIL NOP 2018-02-25
CLE SAS 2018-02-25
WAS PHI 2018-02-25
DEN HOU 2018-02-25
ATL LAL 2018-02-26
BOS MEM 2018-02-26
BRK CHI 2018-02-26
NYK GSW 2018-02-26
TOR DET 2018-02-26
NOP PHO 2018-02-26
OKC ORL 2018-02-26
DAL IND 2018-02-26
UTA HOU 2018-02-26
SAC MIN 2018-02-26
CHO CHI 2018-02-27
CLE BRK 2018-02-27
MIA PHI 2018-02-27
MIL WAS 2018-02-27
POR SAC 2018-02-27
DEN LAC 2018-02-27
DET MIL 2018-02-28
ORL TOR 2018-02-28
ATL IND 2018-02-28
BOS CHO 2018-02-28
WAS GSW 2018